<a href="https://colab.research.google.com/github/ryukya/AI_2020/blob/SVM/Heart_SVM_and_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data preparation

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

##load data
df_heart = pd.read_csv('heart.csv')

##transform value
lb=LabelEncoder()
df_heart=df_heart.apply(lb.fit_transform)
datas=df_heart.rename(columns={ "target": "lab"}, errors="raise")

datas.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,lab
0,29,1,3,31,64,1,0,49,0,22,0,0,1,1
1,3,1,2,22,80,0,1,84,0,32,0,0,2,1
2,7,0,1,22,35,0,0,71,0,14,2,0,2,1
3,22,1,1,14,67,0,1,76,0,8,2,0,2,1
4,23,0,0,14,145,0,1,62,1,6,2,0,2,1


In [0]:
##separate label and features data
dt_label = datas['lab']
df_heart = datas.drop(['lab'],axis=1)
##put data to Y 
Y = []
for val in dt_label:
    if(val == 1): ## val=1 there is a presence of heart diseases
        Y.append(-1)
    else:
        Y.append(1)
print(df_heart.head())

   age  sex  cp  trestbps  chol  fbs  ...  thalach  exang  oldpeak  slope  ca  thal
0   29    1   3        31    64    1  ...       49      0       22      0   0     1
1    3    1   2        22    80    0  ...       84      0       32      0   0     2
2    7    0   1        22    35    0  ...       71      0       14      2   0     2
3   22    1   1        14    67    0  ...       76      0        8      2   0     2
4   23    0   0        14   145    0  ...       62      1        6      2   0     2

[5 rows x 13 columns]


In [0]:
##put data to X
X = df_heart.values.tolist()

## prepare variable
x_train = []
y_train = []
x_test = []
y_test = []

##split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8,random_state=10)

##data leng for  data training and testing
lengt=len(df_heart)
train_len= int(0.8*lengt)
test_len=int( 0.2*lengt)+1

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

y_train = y_train.reshape(train_len,1)
y_test = y_test.reshape(test_len,1)
print("length: ",lengt,"---- training:",train_len,"----test:",test_len)




length:  303 ---- training: 242 ----test: 61


##Evaluation Function

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
def evaluate (y_test,predictions):
  print ("Accuracy Score")
  print(accuracy_score(y_test,predictions))
  print ("Confusion Matrix")
  print(confusion_matrix(y_test, predictions))
  print ("Classification report")
  print(classification_report(y_test, predictions))

##SVM Function

In [0]:
def svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc):
  training=[]
  ##train_fx & training reshape 
  for col in range(x_train.shape[1]):
    training.append(x_train[:,col].reshape(train_len,1))
  ##make w array (zeros) 
  w_zeros=[]
  for x in training:
    w_zeros.append(np.zeros((train_len,1)))
  epochs = 1
  alpha = alpa
  while(epochs < epoc):
    ##sum of (array in w_zeros*array in training)
    sumval = None 
    for xw,yt in zip(w_zeros, training):
      xy=xw*yt
      if sumval is None:
       y1=xy
      else:
        y1= y1 + xy
      sumval =y1 
    prod = sumval * y_train
    count = 0
    for val in prod:
      if(val >= 1):
        cost = 0
        for i in range(len(w_zeros)):
         w_zeros[i]= w_zeros[i] - alpha * (2 * 1/epochs * w_zeros[i])
      else:
        cost = 1 - val
        for i in range(len(w_zeros)):
         w_zeros[i]= w_zeros[i] + alpha * (training[i][count] * y_train[count] - 2 * 1/epochs * w_zeros[i])
      count += 1
    epochs += 1
  ## Clipping weights  and reshape array
  index = list(range(test_len,train_len))
  for i in range(len(w_zeros)):
    w_zeros[i]=np.delete(w_zeros[i],index)
    w_zeros[i] = w_zeros[i].reshape(test_len,1)
  ## Extract the test data features 
  testing=[]
  for col in range(x_test.shape[1]):
    testing.append(x_test[:,col].reshape(test_len,1))
  ## Predict
  pred_val = None 
  for xw,yt in zip(w_zeros, testing):
    xy=xw*yt
    if pred_val is None:
      y1=xy
    else:
      y1= y1 + xy
    pred_val =y1 
  #add prediction result into list
  predictions = []
  for val in pred_val:
      if(val > 1):
          predictions.append(1)
      else:
          predictions.append(-1)
  evaluate(y_test,predictions)     



##All Features SVM

###SVM  with alpha= 0.002 and epoch =500

In [0]:
alpa=0.002
epoc=500
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5737704918032787
Confusion Matrix
[[26  0]
 [26  9]]
Classification report
              precision    recall  f1-score   support

          -1       0.50      1.00      0.67        26
           1       1.00      0.26      0.41        35

    accuracy                           0.57        61
   macro avg       0.75      0.63      0.54        61
weighted avg       0.79      0.57      0.52        61



###SVM  with alpha= 0.001 and epoch =1000

In [0]:
alpa=0.001
epoc=1000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.6065573770491803
Confusion Matrix
[[26  0]
 [24 11]]
Classification report
              precision    recall  f1-score   support

          -1       0.52      1.00      0.68        26
           1       1.00      0.31      0.48        35

    accuracy                           0.61        61
   macro avg       0.76      0.66      0.58        61
weighted avg       0.80      0.61      0.57        61



###SVM  with alpha= 0.0005 and epoch =2000

In [0]:
alpa=0.0005
epoc=2000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.6229508196721312
Confusion Matrix
[[26  0]
 [23 12]]
Classification report
              precision    recall  f1-score   support

          -1       0.53      1.00      0.69        26
           1       1.00      0.34      0.51        35

    accuracy                           0.62        61
   macro avg       0.77      0.67      0.60        61
weighted avg       0.80      0.62      0.59        61



###SVM  with alpha= 0.0002 and epoch =5000

In [0]:
alpa=0.0002
epoc=5000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.6557377049180327
Confusion Matrix
[[26  0]
 [21 14]]
Classification report
              precision    recall  f1-score   support

          -1       0.55      1.00      0.71        26
           1       1.00      0.40      0.57        35

    accuracy                           0.66        61
   macro avg       0.78      0.70      0.64        61
weighted avg       0.81      0.66      0.63        61



###SVM  with alpha= 0.0001 and epoch =10000

In [0]:
alpa=0.0001
epoc=10000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.7540983606557377
Confusion Matrix
[[24  2]
 [13 22]]
Classification report
              precision    recall  f1-score   support

          -1       0.65      0.92      0.76        26
           1       0.92      0.63      0.75        35

    accuracy                           0.75        61
   macro avg       0.78      0.78      0.75        61
weighted avg       0.80      0.75      0.75        61



###Result if Using Scikit SVM Library

In [0]:
from sklearn import svm
sv_f = svm.SVC(kernel='linear') # Linear Kernel
sv_f.fit(x_train, y_train)
y_pred = sv_f.predict(x_test)
evaluate(y_test, y_pred)


Accuracy Score
0.7704918032786885
Confusion Matrix
[[21  5]
 [ 9 26]]
Classification report
              precision    recall  f1-score   support

          -1       0.70      0.81      0.75        26
           1       0.84      0.74      0.79        35

    accuracy                           0.77        61
   macro avg       0.77      0.78      0.77        61
weighted avg       0.78      0.77      0.77        61



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##Removing Random Features ( 50% of Total Features)

In [0]:
from random import seed
from random import randint

seed(1)
cols = []
col_tot= len(df_heart.columns)-1
half=int(col_tot*0.5)
for _ in range(half):
	value = randint(0, col_tot)
	cols.append(value)
print (cols)

df_heart.drop(df_heart.columns[cols],axis=1,inplace=True)
df_heart.shape

[2, 9, 12, 12, 1, 4]


(303, 8)

### Data preaparation for half features data




In [0]:

##put data to X
X = df_heart.values.tolist()

## prepare variable
x_train = []
y_train = []
x_test = []
y_test = []

##split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=10)

##data leng for  data training and testing using from the first dat apreparation


x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

y_train = y_train.reshape(train_len,1)
y_test = y_test.reshape(test_len,1)

x_train.shape


(242, 8)

###SVM with alpha= 0.002 and epoch =500

In [0]:
alpa=0.002
epoc=500
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.4262295081967213
Confusion Matrix
[[25  1]
 [34  1]]
Classification report
              precision    recall  f1-score   support

          -1       0.42      0.96      0.59        26
           1       0.50      0.03      0.05        35

    accuracy                           0.43        61
   macro avg       0.46      0.50      0.32        61
weighted avg       0.47      0.43      0.28        61



###SVM  with alpha= 0.001 and epoch =1000

In [0]:
alpa=0.001
epoc=1000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5081967213114754
Confusion Matrix
[[25  1]
 [29  6]]
Classification report
              precision    recall  f1-score   support

          -1       0.46      0.96      0.62        26
           1       0.86      0.17      0.29        35

    accuracy                           0.51        61
   macro avg       0.66      0.57      0.46        61
weighted avg       0.69      0.51      0.43        61



###SVM with alpha= 0.0005 and epoch =2000

In [0]:
alpa=0.0005
epoc=2000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5409836065573771
Confusion Matrix
[[25  1]
 [27  8]]
Classification report
              precision    recall  f1-score   support

          -1       0.48      0.96      0.64        26
           1       0.89      0.23      0.36        35

    accuracy                           0.54        61
   macro avg       0.68      0.60      0.50        61
weighted avg       0.71      0.54      0.48        61



###SVM  with alpha= 0.0002 and epoch =5000

In [0]:
alpa=0.0002
epoc=5000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.5245901639344263
Confusion Matrix
[[25  1]
 [28  7]]
Classification report
              precision    recall  f1-score   support

          -1       0.47      0.96      0.63        26
           1       0.88      0.20      0.33        35

    accuracy                           0.52        61
   macro avg       0.67      0.58      0.48        61
weighted avg       0.70      0.52      0.46        61



###SVM  with alpha= 0.0001 and epoch =10000

In [0]:
alpa=0.0001
epoc=10000
svm_f( x_train, x_test, y_train, y_test, train_len,test_len,alpa,epoc)

Accuracy Score
0.6065573770491803
Confusion Matrix
[[24  2]
 [22 13]]
Classification report
              precision    recall  f1-score   support

          -1       0.52      0.92      0.67        26
           1       0.87      0.37      0.52        35

    accuracy                           0.61        61
   macro avg       0.69      0.65      0.59        61
weighted avg       0.72      0.61      0.58        61



###Result if Using Scikit SVM Library

In [0]:
sv_f = svm.SVC(kernel='linear') # Linear Kernel
sv_f.fit(x_train, y_train)
y_pred = sv_f.predict(x_test)
evaluate(y_test, y_pred)


Accuracy Score
0.7049180327868853
Confusion Matrix
[[19  7]
 [11 24]]
Classification report
              precision    recall  f1-score   support

          -1       0.63      0.73      0.68        26
           1       0.77      0.69      0.73        35

    accuracy                           0.70        61
   macro avg       0.70      0.71      0.70        61
weighted avg       0.71      0.70      0.71        61



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
